In [1]:
!nvidia-smi

Tue Nov 17 05:14:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   37C    P8    26W / 260W |   7203MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 27%   31C    P8     9W / 260W |   6215MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import pandas as pd
import numpy as np
import pickle

from pathlib import Path

from box import Box
import pandas as pd
import collections
import os
import sys
import random

import datetime

from transformers import AutoTokenizer

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner

import re
MAX_SENTENCE_LEN = 82

from utils import preprocess
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

import torch
import torchtext
import torchtext.data as data
from torchtext.data import Field
from torchtext.data import Iterator, BucketIterator
from transformers import BertTokenizer, BertForMaskedLM, AdamW


from matplotlib import pyplot as plt
%matplotlib inline

from functools import partial
import time
from IPython.core.debugger import set_trace
from tqdm import tqdm, trange
# from tqdm import tqdm, trange

seed = 678

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
torch.cuda.set_device(7)
cuda = torch.cuda.is_available()

## Roberta

Loading data.

In [4]:
from ast import literal_eval

DATA_PATH = '../../../semeval2021_task/data/'

train = pd.read_csv(DATA_PATH + 'tsd_train.csv')
trial = pd.read_csv(DATA_PATH + 'tsd_trial.csv')

train['spans'] = train.spans.apply(literal_eval)
trial['spans'] = trial.spans.apply(literal_eval)
texts = list(train['text'])

Loading model.

In [5]:
CLASSIFICATION_MODEL_PATH = '../../models/roberta_full/model_out'
CLASSIFICATION_LABELS_PATH = '../../labels'

from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor(CLASSIFICATION_MODEL_PATH, CLASSIFICATION_LABELS_PATH, 
                                        multi_label=True, model_type='roberta')

Some weights of the model checkpoint at ../../models/roberta_full/model_out were not used when initializing RobertaForMultiLabelSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
lern = predictor.get_learner()
pretrained_weights = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
MASK_INDEX = tokenizer.convert_tokens_to_ids("[MASK]")

lern.model.eval();

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Some weights of the model checkpoint at ../../models/roberta_full/model_out were not used when initializing RobertaForMultiLabelSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Device: cuda


In [201]:
v = {v: k for k, v in tokenizer.get_vocab().items()}

def toks_to_words(token_ids):
    """ Merge subword tokens into whole words """
    indices = []
    for i, token_id in enumerate(token_ids):
        if i == 0:
            continue
        token_text = v[token_id]
        if not token_text.startswith('Ġ') and token_text.isalpha() and i != 1:
            indices.append(i)
        else:
            if indices:
                toks = [v[token_ids[t]] for t in indices]
                word = ''.join(toks)
                new_indices = [index - 1 for index in indices]
                yield new_indices, word
            indices = [i]

In [202]:
from transformers import RobertaConfig
config = RobertaConfig.from_pretrained('roberta-base', output_attentions=True)
lern.model.roberta.config = config

Some useful functions

In [203]:
def shrink(nums_arr, att_arr):
    """
    When words are splitted by several tokens by BERT, each token has
    its own feature vector. this function takes the mean of these vectors
    and assign it to the original word.
    
    """
    res = []
    prev_i = 0
    #print(att_arr.shape)
    for arr in nums_arr:#for each splitted word
        fig = att_arr[:, :, prev_i:arr[0]] #add all previous words
        nafig = att_arr[:, :, arr].mean(axis=2)[:, :, np.newaxis] #add mean of token features
        #print(fig.shape)
        #print(nafig.shape)
        res.append(fig)
        res.append(nafig)
        prev_i = arr[-1] + 1 #make the current word "previous"
        #print(prev_i, att_arr.shape[-1])
    #print("before checker")
    #for fig in res:
        #print(fig.shape)
    if prev_i < att_arr.shape[-1]: #just to be safe and not overstep the array
        res.append(att_arr[:, :, prev_i:])
    #print("after checker")
    #for fig in res:
        #print(fig.shape)
    result = np.concatenate(res, axis=2).T
    #print(result.shape)
    return result

In [209]:
def get_attention_for_words(text: str, model, tokenizer):
    """
    Get attention values for all heads averaged over all layers and 
    attention values for all layers averaged over all heads.
    """
    toks = tokenizer.encode(text) 
    nums = [arr for (arr, string) in list(toks_to_words(toks)) if len(arr) > 1]
    #print(nums)
    sentence = torch.tensor(toks).unsqueeze(0)
    sentence = sentence.cuda()
    out = model.roberta(sentence, output_attentions=True, return_dict=True)
    #we cut the CLS and SEP tokens
    attentions = torch.cat(out['attentions'], dim=0).cpu()[:, :, 1:-1, 1:-1]
    attentions = attentions.mean(dim=2).detach().numpy()
    #print(attentions.shape)
    shrunk_attentions = shrink(nums, attentions).reshape(-1, 144)
    return shrunk_attentions

In [210]:
def getword(arr, text):
    """
    Using a span from a dataset, obtain a word
    """
    ans = ''
    for i in range(len(arr)):
        elem = arr[i]
        if  i != 0 and i != len(arr) - 1 and elem != arr[i-1] + 1:
            ans += ' '
            ans += text[elem]
        else:
            ans += text[elem]
    return ans

In [211]:
def make_target(span, text, tokenizer):
    target_toks = tokenizer.encode(getword(span, text))
    #print(target_toks)
    toks = tokenizer.encode(text)
    #print(toks)
    toks = [string for (arr, string) in list(toks_to_words(toks))]
    #print(toks)
    target_toks = [string for (arr, string) in list(toks_to_words(target_toks))]
    #print(target_toks)
    target = []
    #check if the word is in spanned words
    for tok in toks:
        if len(target_toks) > 0 and tok.strip('Ġ') == target_toks[0].strip('Ġ'):
            target.append(1)
            target_toks = target_toks[1:]
        else:
            target.append(0)
    return toks, target
        

In [212]:
make_target(train.spans.iloc[0], train.text.iloc[0], tokenizer)

(['Another',
  'Ġviolent',
  'Ġand',
  'Ġaggressive',
  'Ġimmigrant',
  'Ġkilling',
  'Ġa',
  'Ġinnocent',
  'Ġand',
  'Ġintelligent',
  'ĠUS',
  'ĠCitizen',
  '....',
  'ĠSarcasm'],
 [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [213]:
len(tokenizer.encode(train.text.iloc[0]))

18

In [174]:
list(toks_to_words(tokenizer.encode(train.text.iloc[0])))

[([0], 'Another'),
 ([1], 'Ġviolent'),
 ([2], 'Ġand'),
 ([3], 'Ġaggressive'),
 ([4], 'Ġimmigrant'),
 ([5], 'Ġkilling'),
 ([6], 'Ġa'),
 ([7], 'Ġinnocent'),
 ([8], 'Ġand'),
 ([9], 'Ġintelligent'),
 ([10], 'ĠUS'),
 ([11], 'ĠCitizen'),
 ([12], '....'),
 ([13, 14, 15], 'ĠSarcasm')]

In [161]:
train.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [214]:
target = []
data = np.array([])
for span, text in tqdm(zip(train['spans'], train['text'])):
    word_attentions = get_attention_for_words(text, lern.model, tokenizer)
    #print(word_attentions.shape)
    if len(data) == 0:
        data = word_attentions
    else:
        data = np.vstack((data, word_attentions))
    words, new_target = make_target(span, text, tokenizer)
    #print(new_target)
    try:
        assert len(new_target) == word_attentions.shape[0]
    except:
        print(new_target)
        print(words)
        print(word_attentions.shape)
        print(len(words))
        print(text)
        break
    target += new_target
    

7939it [07:41, 17.20it/s]


Classification

In [215]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(data, target)

/home/markov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [217]:
trial_data = np.array([])
for text in tqdm(trial['text']):
    word_atts = get_attention_for_words(text, lern.model, tokenizer)
    if len(trial_data) == 0:
        trial_data = word_atts
    else:
        trial_data = np.vstack((trial_data, word_atts))
    

100%|██████████| 690/690 [00:12<00:00, 57.30it/s]


In [218]:
train_preds = logreg.predict(data)

In [219]:
train_preds.sum()

6838

In [221]:
sum(target)

21978

In [225]:
train_probas = logreg.predict_proba(data)[:, 1]

In [227]:
print(len(train_probas))

334930


In [229]:
for i in range(10):
    train_pred = np.where(train_probas > i/10, 1, 0)
    print(train_pred.sum(), i)

334930 0
25072 1
13038 2
9742 3
8045 4
6838 5
5915 6
5114 7
4260 8
3230 9


In [257]:
trial_probas = logreg.predict_proba(trial_data)[:, 1]
trial_preds = np.where(trial_probas > 0.18, 1, 0)

In [258]:
trial_target = []
for span, text in tqdm(zip(trial['spans'], trial['text'])):
    words, new_target = make_target(span, text, tokenizer)
    trial_target += new_target
    

690it [00:00, 2136.52it/s]


In [259]:
trial_preds.sum()

1239

In [260]:
sum(trial_target)

1508

In [261]:
from sklearn.metrics import accuracy_score, f1_score
print(f1_score(trial_preds, trial_target))

0.5242082271568983
